In [1]:
import pandas as pd
import numpy as np
# from google.colab import drive
# drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/exams.csv')
df.head()

Mounted at /content/drive


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group D,some college,standard,completed,59,70,78
1,male,group D,associate's degree,standard,none,96,93,87
2,female,group D,some college,free/reduced,none,57,76,77
3,male,group B,some college,free/reduced,none,70,70,63
4,female,group D,associate's degree,standard,none,83,85,86


In [2]:
# For Gender: unprivileged group: female; priviliged group: male;
# For Race: unprivileged group: Group A B % E; priviliged group: Group C and D;
# Fairness metrics:
# Statistical Parity Difference
# Disparate Impact
# If math score >= 60 and writing score >= 60, this will be the favorable outcome; if math score < 60 and writing score < 60, this will be the unfavorable outcome.

male_math_favorable, female_math_favorable, male_writing_favorable, female_writing_favorable, male_math_unfavorable, female_math_unfavorable, male_writing_unfavorable, female_writing_unfavorable = 0, 0, 0, 0, 0, 0, 0, 0
groupCD_math_favorable, groupABE_math_favorable, groupCD_writing_favorable, groupABE_writing_favorable, groupCD_math_unfavorable, groupABE_math_unfavorable, groupCD_writing_unfavorable, groupABE_writing_unfavorable = 0, 0, 0, 0, 0, 0, 0, 0


In [7]:
for index, row in df.iterrows():
  if row['gender'] == 'male':
    if row['math score'] >= 60:
      male_math_favorable += 1
    else:
      male_math_unfavorable += 1
    if row['writing score'] >= 60:
      male_writing_favorable += 1
    else:
      male_writing_unfavorable += 1
  else:
    if row['math score'] >= 60:
      female_math_favorable += 1
    else:
      female_math_unfavorable += 1
    if row['writing score'] >= 60:
      female_writing_favorable += 1
    else:
      female_writing_unfavorable += 1
  if row['race/ethnicity'] == 'group C' or row['race/ethnicity'] == 'group D':
    if row['math score'] >= 60:
      groupCD_math_favorable += 1
    else:
      groupCD_math_unfavorable +=1
    if row['writing score'] >= 60:
      groupCD_writing_favorable += 1
    else:
      groupCD_writing_unfavorable +=1
  else:
    if row['math score'] >= 60:
      groupABE_math_favorable += 1
    else:
      groupABE_math_unfavorable +=1
    if row['writing score'] >= 60:
      groupABE_writing_favorable += 1
    else:
      groupABE_writing_unfavorable +=1

In [8]:
og_gender_math_disparate_impact = (female_math_favorable / (female_math_favorable + female_math_unfavorable)) / (male_math_favorable / (male_math_favorable + male_math_unfavorable))
og_gender_writing_disparate_impact = (female_writing_favorable / (female_writing_favorable + female_writing_unfavorable)) / (male_writing_favorable / (male_writing_favorable + male_writing_unfavorable))
og_race_math_disparate_impact = (groupABE_math_favorable / (groupABE_math_favorable + groupABE_math_unfavorable)) / (groupCD_math_favorable / (groupCD_math_favorable + groupCD_math_unfavorable))
og_race_writing_disparate_impact = (groupABE_writing_favorable / (groupABE_writing_favorable + groupABE_writing_unfavorable)) / (groupCD_writing_favorable / (groupCD_writing_favorable + groupCD_writing_unfavorable))

og_gender_math_statistical_parity = (female_math_favorable / (female_math_favorable + female_math_unfavorable)) - (male_math_favorable / (male_math_favorable + male_math_unfavorable))
og_gender_writing_statistical_parity = (female_writing_favorable / (female_writing_favorable + female_writing_unfavorable)) - (male_writing_favorable / (male_writing_favorable + male_writing_unfavorable))
og_race_math_statistical_parity = (groupABE_math_favorable / (groupABE_math_favorable + groupABE_math_unfavorable)) - (groupCD_math_favorable / (groupCD_math_favorable + groupCD_math_unfavorable))
og_race_writing_statistical_parity = (groupABE_writing_favorable / (groupABE_writing_favorable + groupABE_writing_unfavorable)) - (groupCD_writing_favorable / (groupCD_writing_favorable + groupCD_writing_unfavorable))

original_metrics = [og_gender_math_disparate_impact,
                   og_gender_writing_disparate_impact,
                   og_race_math_disparate_impact,
                   og_race_writing_disparate_impact,
                   og_gender_math_statistical_parity,
                   og_gender_writing_statistical_parity,
                   og_race_math_statistical_parity,
                   og_race_writing_statistical_parity]
independent_lables = ['gender', 'gender', 'race', 'race', 'gender', 'gender', 'race', 'race']
dependent_lables = ['math score', 'writing score', 'math score', 'writing score', 'math score', 'writing score', 'math score', 'writing score']
metric_names = ['disparate impact', 'disparate impact', 'disparate impact', 'disparate impact', 'statistical parity', 'statistical parity', 'statistical parity', 'statistical parity']

og_summary_table = list(zip(independent_lables, dependent_lables, metric_names, original_metrics))
og_summary_df = pd.DataFrame(og_summary_table, columns = ['Indepedent Variable', 'Dependent Variable', 'Fairness Metric', 'Original Metric Value'])
og_summary_df.head(10)

,Indepedent Variable,Dependent Variable,Fairness Metric,Original Metric Value
0,gender,math score,disparate impact,0.875108
1,gender,writing score,disparate impact,1.269594
2,race,math score,disparate impact,0.962202
3,race,writing score,disparate impact,0.973943
4,gender,math score,statistical parity,-0.095144
5,gender,writing score,statistical parity,0.175661
6,race,math score,statistical parity,-0.027616
7,race,writing score,statistical parity,-0.019518
